In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
feature_x_sample = pd.read_table(
    PATH_DICT['feature_x_sample_file_path'],
    index_col=0,
)

In [ ]:
for element in ELEMENTS:
    
    if element == 'feature':
        
        df = feature_x_sample
        
    elif element == 'sample':
        
        df = feature_x_sample.T
        
    skew_t_pdf_fit_parameter = ccal.fit_skew_t_pdfs(
        df,
        n_job=MAX_N_JOB,
        directory_path=PATH_DICT['{}_directory_path'.format(element)],
    )
    
#     skew_t_pdf_fit_parameter = pd.read_table(
#         '{}/skew_t_pdf_fit_parameter.tsv'.format(PATH_DICT['{}_directory_path'.format(element)]),
#         index_col=0,
#     )
    
    skew_t_pdf_fit_parameter['log(Degree of Freedom)'] = ccal.log_nd_array(skew_t_pdf_fit_parameter['Degree of Freedom'])
    
    for parameter_name, parameter_values in skew_t_pdf_fit_parameter.drop(
        'Degree of Freedom',
        axis=1,
    ).items():
        
        value_name = '{} {}'.format(
            element.title(),
            parameter_name,
        )
        
        ccal.plot_distributions(
            (parameter_values, ),
            names=(parameter_name, ),
            plot_rug=False,
            title='{} Distribution'.format(value_name),
            xaxis_title=value_name,
        )
        
        parameter_values_sorted = parameter_values.sort_values()
        
        index_min = parameter_values_sorted.index[0]
        
        index_max = parameter_values_sorted.index[-1]
        
        ccal.plot_distributions(
            (
                df.loc[index_min].dropna(),
                df.loc[index_max].dropna(),
            ),
            names=(
                index_min,
                index_max,
            ),
            plot_rug=False,
            title='{} Extreme {} Distibutions'.format(
                element.title(),
                parameter_name,
            ),
            xaxis_title='Value',
        )
        
        n_extreme = 3
        
        for rank in list(range(n_extreme)) + list(range(
            -n_extreme,
            0,
        )):
            
            index = parameter_values_sorted.index[rank]
            
            parameter_values = df.loc[index]
            
            n_data, location, scale, degree_of_freedom, shape = skew_t_pdf_fit_parameter.loc[index, [
                'N Data',
                'Location',
                'Scale',
                'Degree of Freedom',
                'Shape',
            ]]
            
            ccal.plot_context(
                parameter_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=parameter_values.dropna().size < 1e3,
                title='{} {} Rank {}: {}'.format(
                    element.title(),
                    parameter_name,
                    rank,
                    index,
                ),
                xaxis_title='Value',
            )